In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json

In [ ]:
book_id = ''
s3 = boto3.resource('s3')

obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)
book = gc.open_by_key(book_id)

In [ ]:
my_key = get_key('default')
all_pubs = ff_utils.search_metadata('/search/?type=Publication&number_of_experiment_sets%21=No+value', key = my_key)

In [ ]:
print(len(all_pubs))

In [ ]:
book

In [ ]:
worksheet = book.worksheet('Publication List')

In [ ]:
table = worksheet.get_all_values()
##Convert table data into a dataframe
df = pd.DataFrame(table[1:], columns=table[0])
col_names = df.columns.values

In [ ]:
titles_in_pub = df['Title']

In [ ]:
for pub in all_pubs:
    print(pub.keys())
    break

In [ ]:
print(sample_pub.keys())

In [ ]:
from difflib import SequenceMatcher as SM

In [ ]:
links_col = []
missing_pubs = []
for pub in all_pubs:
    display_title = pub['display_title']
    title = pub['title']
    found = False
    for index, title_in_gs in enumerate(df['Title']):
        formatted_link = ''
        match_score = SM(None, title_in_gs, title).ratio()
        if match_score >= 0.90:
            num_exps = pub['number_of_experiment_sets'] 
            link_to_exps = 'https://data.4dnucleome.org/browse/?experimentset_type=replicate&publications_of_set.display_title={}'.format(display_title)
            formatted_link = '=hyperlink("{}", "{} expsets in portal")'.format(link_to_exps, num_exps)
            found = True
            col_link = [index, formatted_link]
            links_col.append(col_link)
    if found is False:
        missing_pubs.append(title)   

In [ ]:
print(len(links_col))

In [ ]:
print(links_col)

In [ ]:
column_df = [''] * len(df['Title']) 

In [ ]:
print(column_df)

In [ ]:
for item in links_col:
    column_df[item[0]] = item[1]

In [ ]:
#print(links_col)

In [ ]:
print(len(missing_pubs))

In [ ]:
all_pubs[0].keys()

In [ ]:
df['Data in Portal'] = column_df

In [ ]:
df['Portal link'] = column_df

In [ ]:
#print(df)

In [ ]:
data = df.values.tolist()

In [ ]:
#df.head()

In [ ]:
df = df[['Title', 'Authors', 'Date', 'Contact 4DN Author', 'Data in Portal', 'priority', 'status', 'Primary_awards', 'Secondary Awards', 'Other ids', 'Link', 'Date added']]

In [ ]:
#df.head()

In [ ]:
worksheet = book.add_worksheet('Publication List test', 100,100)

In [ ]:
labels = ['Title', 'Authors', 'Date', 'Contact 4DN Author', 'Portal link', 'Priority', 'Status', 'Primary awards', 'Secondary awards', 'Other ids', 'Publication Link', 'Date added']
gs_write = []
row = 1
col = 1
for label in labels:
    gs_write.append(gspread.models.Cell(1,col,label))
    col = col + 1

col = 1
for item in data:
    row = row + 1
    for ele in item:
        gs_write.append(gspread.models.Cell(row,col,ele))
        col = col + 1
    col = 1

worksheet.update_cells(gs_write, value_input_option='USER_ENTERED')

In [ ]:
###############################################################################################################

In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
from difflib import SequenceMatcher as SM
import time

In [ ]:
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

In [ ]:
my_key = get_key('default')
all_pubs = ff_utils.search_metadata('/search/?type=Publication&number_of_experiment_sets%21=No+value', key=my_key)

In [ ]:
#Get all the centers info from google management sheets
gs_management_sheet_id = ''
gs_management_workbook = gc.open_by_key(gs_management_sheet_id)
gs_management_worksheet = gs_management_workbook.sheet1

centers_sheets_links = gs_management_worksheet.get_all_values()

In [ ]:
for center in centers_sheets_links:
    if center[1] != "1U01EB021223-01":
        continue
    print(center)

In [ ]:
all_missing_pubs = []
all_matched_pubs = []
ignore_first_col = 0
request_ct = 0
for center in centers_sheets_links:
#     if center[1] != "1U01EB021223-01":
#         continue
    print(center[0])
    if ignore_first_col == 0:
        ignore_first_col = ignore_first_col + 1
        continue
    request_ct = request_ct + 1
    grant_gs_id = center[2]
    if request_ct == 15:
        time.sleep(100)
        request_ct = 0
    workbook = gc.open_by_key(grant_gs_id)
    print('working on', workbook)
    try:
        worksheet = workbook.worksheet('Publication List')
    except:
        continue
    
    table = worksheet.get_all_values()
    ##Convert table data into a dataframe
    df = pd.DataFrame(table[1:], columns=table[0])
    col_names = df.columns.values
    
    links_col = []
    missing_pubs = []
    matched_pubs = []

    for pub in all_pubs:
        display_title = pub['display_title']
        title = pub['title']
        found = False
        for index, title_in_gs in enumerate(df['Title']):
            formatted_link = ''
            match_score = SM(None, title_in_gs, title).ratio()
            if match_score >= 0.90:
                num_exps = pub['number_of_experiment_sets'] 
                link_to_exps = 'https://data.4dnucleome.org/browse/?experimentset_type=replicate&publications_of_set.display_title={}'.format(display_title)
                formatted_link = '=hyperlink("{}", "{} expsets in portal")'.format(link_to_exps, num_exps)
                found = True
                col_link = [index, formatted_link]
                links_col.append(col_link)
                matched_pubs.append(title)
        if found is False:
            missing_pubs.append(title)
    
    print(len(links_col), 'were found')
    all_missing_pubs = all_missing_pubs + missing_pubs
    all_matched_pubs = all_matched_pubs + matched_pubs
    print()
    
    column_df = [''] * len(df['Title']) 
    for item in links_col:
        column_df[item[0]] = item[1]
    df['Portal link'] = column_df
    
#     # Arrange columns in the right order
#     col_order_first = ['Title', 'Authors', 'Date', 'Contact 4DN Author', 'Portal link']
#     col_order_middle = []
#     col_order_last = ['Primary_awards', 'Secondary Awards', 'Other ids','Link','Date added']
    
#     for name in col_names:
#         if name in col_order_first or name in col_order_last:
#             continue
#         else:
#             col_order_middle.append(name)
#     col_order = col_order_first + col_order_middle + col_order_last
#     try:
#         df = df[col_order]
#     except:
#         print('already exists')
#         continue
    
#     labels_first = ['Title', 'Authors', 'Date', 'Contact 4DN Author', 'Portal link']
#     labels_middle = col_order_middle
#     labels_last = ['Primary awards', 'Secondary awards', 'Other ids','Publication link','Date added']
#     labels = labels_first + labels_middle + labels_last
       
    data = df.values.tolist()
       
    gs_write = []
    row = 1
    col = 1
#     for label in labels:
#         gs_write.append(gspread.models.Cell(1,col,label))
#         col = col + 1

    col = 1
    for item in data:
        row = row + 1
        for ele in item:
            gs_write.append(gspread.models.Cell(row,col,ele))
            col = col + 1
        col = 1
        
    if request_ct == 15:
        time.sleep(100)
        request_ct = 0
        
    worksheet.update_cells(gs_write, value_input_option='USER_ENTERED')

In [ ]:
print(log)

In [ ]:
print(list(set(log[0])))

In [ ]:
print(len(log[0]))

In [ ]:
print(log)

In [ ]:
x = 0
for pub in all_pubs:
    if pub['title'] in log[0]:
        log[0].pop(x)
    x = x + 1

In [ ]:
#print(len(log))

In [ ]:
print(len(all_matched_pubs))

In [ ]:
print(len(all_missing_pubs))

In [ ]:
unique_all_matched_pubs = set(all_matched_pubs)

In [ ]:
unique_all_missing_pubs = set(all_missing_pubs)

In [ ]:
print(len(unique_all_matched_pubs))

In [ ]:
print(len(unique_all_missing_pubs))

In [ ]:
intersection = unique_all_missing_pubs.intersection(unique_all_matched_pubs)

In [ ]:
len(intersection)

In [ ]:
total_missing_pubs = [x  for x in list(unique_all_missing_pubs) if x not in list(intersection)]

In [ ]:
print(len(total_missing_pubs ))

In [ ]:
print(total_missing_pubs)

In [ ]:
print(unique_all_matched_pubs)